### Reading Data and getting a list of all the chemicals

In [1]:
import pandas as pd
import numpy as np

In [2]:
boiling_data = pd.read_excel('BoilingPointData.xlsx',index_col = 0)
boiling_data.head()

,name,molweight,critical temperature (K),acentric factor,boiling point (K)
1,(+)-a-pinene,136.23704,647.00,0.341,428.65
2,(+)-camphene,136.23704,638.00,0.296,432.65
3,(-)-a-pinene,136.23704,647.00,0.341,429.35
4,(-)-b-citronellol,156.26820,656.59,0.612,498.65
5,(-)-camphene,136.23704,638.00,0.296,439.95


### Trying different methods of converting from chemical name to SMILES
#### From research, it was found that the main RDKit cannot convert IUPAC names to SMILEs

#### Method 1: Using Webscraping method (from CIR: Chemical Identifier Resolver)
Link: https://cactus.nci.nih.gov/chemical/structure

In [3]:
from urllib.request import urlopen
from urllib.parse import quote

In [4]:
#A copy and pasted 

def CIRconvert(ids):
    try:
        url = 'http://cactus.nci.nih.gov/chemical/structure/' + quote(ids) + '/smiles'
        ans = urlopen(url).read().decode('utf8')
        return ans
    except:
        return 'Did not work'

identifiers  = ['(S)-(+)-1-amino-2-propanol','(<+->)-2-chlorooctane', '(R)-2-amino-6-methylheptane', '(E)-2,2,5,5-tetramethylhex-3-ene', 'Diethyl sulfate', "1,1'-[methylenebis(thio)]bisethane", '(1-ethyloctadecyl)benzene']

for ids in identifiers :
    
    print(CIRconvert(ids))

C[C@H](O)CN
Did not work
CC(C)CCC[C@@H](C)N
CC(C)(C)\C=C\C(C)(C)C
CCO[S](=O)(=O)OCC
CCSCSCC
CCCCCCCCCCCCCCCCCC(CC)c1ccccc1


In [5]:
#Adapt to my needs: Record down all chemicals that don't work and measure run time.
def CIRconvert_adapted(df,col = 'name'):
    """Inputs chemical data dataframe (and the column name for IUPAC names) and returns a new dataframe with an added Column:
       SMILES. Uses the online CIR API to convert IUPAC names to SMILES strings"""
    data = df
    data['SMILES'] =''
    for i, id in enumerate(df[str(col)],1):
        try:
            url = 'http://cactus.nci.nih.gov/chemical/structure/' + quote(id) + '/smiles'
            ans = urlopen(url).read().decode('utf8')
            data.at[i,'SMILES'] = str(ans)
            print(f"{id} : {ans}")
        except:
            data.at[i,'SMILES'] = str('-')
            print(f"{id} did not work")
            
    return data

Now that we see this works, let's put in the whole chemical data base. With timing

In [6]:
import time

# Grab Currrent Time Before Running the Code
start = time.time()

data_SMILES = CIRconvert_adapted(boiling_data)

end = time.time()

#Subtract Start Time from The End Time
total_time = end - start
print("\n"+ str(total_time))
#First time took 12000 seconds

(+)-a-pinene did not work
(+)-camphene : CC1(C)C2CCC(C2)C1=C
(-)-a-pinene did not work
(-)-b-citronellol did not work
(-)-camphene : CC1(C)C2CCC(C2)C1=C
(1,1-dimethylbutyl)benzene : CCCC(C)(C)c1ccccc1
(1-butylhexadecyl)benzene : CCCCCCCCCCCCCCCC(CCCC)c1ccccc1
(1-ethyl-1-methylpropyl)benzene : CCC(C)(CC)c1ccccc1
(1-ethylbutyl)benzene : CCCC(CC)c1ccccc1
(1-ethyloctadecyl)benzene : CCCCCCCCCCCCCCCCCC(CC)c1ccccc1
(1-hexylheptyl)benzene : CCCCCCC(CCCCCC)c1ccccc1
(1-methylenepropyl)benzene : CCC(=C)c1ccccc1
(1-methylheptyl)benzene : CCCCCCC(C)c1ccccc1
(1-methylnonadecyl)benzene : CCCCCCCCCCCCCCCCCCC(C)c1ccccc1
(1-methylnonyl)benzene : CCCCCCCCC(C)c1ccccc1
(1-methylpentyl)benzene : CCCCC(C)c1ccccc1
(1-octyldodecyl)benzene : CCCCCCCCCCCC(CCCCCCCC)c1ccccc1
(1-propylheptadecyl)benzene : CCCCCCCCCCCCCCCCC(CCC)c1ccccc1
(1-thiapropyl)-benzene : CCSc1ccccc1
(1R)-(-)-menthyl chloride : CC(C)C1CC[C@@H](C)CC1Cl
(1R,2S,5R)-(-)-menthol : CC(C)[C@@H]1CC[C@@H](C)C[C@H]1O
(1S)-(-)-b-pinene did not work
(2,4

1,1,2,2-tetrachloroethane : ClC(Cl)C(Cl)Cl
1,1,2,2-tetrachloropropane : CC(Cl)(Cl)C(Cl)Cl
1,1,2,2-tetrafluoroethane : FC(F)C(F)F
1,1,2,2-tetrafluoropropane : CC(F)(F)C(F)F
1,1,2,2-tetraiodoethane : IC(I)C(I)I
1,1,2,2-tetraiodopropane : CC(I)(I)C(I)I
1,1,2,2-tetramethylcyclopentane : CC1(C)CCCC1(C)C
1,1,2,2-tetraphenylethane : c1ccc(cc1)C(C(c2ccccc2)c3ccccc3)c4ccccc4
1,1,2,3,3,4,4,4-octafluoro-1-butene : FC(F)=C(F)C(F)(F)C(F)(F)F
1,1,2,3,3-pentafluoropropane : FC(F)C(F)C(F)F
1,1,2,3-tetrabromopropane : BrCC(Br)C(Br)Br
1,1,2,3-tetrachloropropane : ClCC(Cl)C(Cl)Cl
1,1,2,3-tetrafluoropropane : FCC(F)C(F)F
1,1,2,3-tetraiodopropane : ICC(I)C(I)I
1,1,2-tribromo-2-methylpropane : CC(C)(Br)C(Br)Br
1,1,2-tribromobutane : CCC(Br)C(Br)Br
1,1,2-tribromoethane : BrCC(Br)Br
1,1,2-tribromopropane : CC(Br)C(Br)Br
1,1,2-trichloro-1,2,2-trifluoroethane : FC(F)(Cl)C(F)(Cl)Cl
1,1,2-trichloro-1,2-difluoroethane : FC(Cl)C(F)(Cl)Cl
1,1,2-trichloro-1-fluoroethane : FC(Cl)(Cl)CCl
1,1,2-trichloro-2-fluoroethane 

1,1-diiodohexane : CCCCCC(I)I
1,1-diiodononane : CCCCCCCCC(I)I
1,1-diiodooctadecane : CCCCCCCCCCCCCCCCCC(I)I
1,1-diiodooctane : CCCCCCCC(I)I
1,1-diiodopentadecane : CCCCCCCCCCCCCCC(I)I
1,1-diiodopentane : CCCCC(I)I
1,1-diiodopropane : CCC(I)I
1,1-diiodotetradecane : CCCCCCCCCCCCCC(I)I
1,1-diiodotridecane : CCCCCCCCCCCCC(I)I
1,1-diiodoundecane : CCCCCCCCCCC(I)I
1,1-dimethyl-2-ethylcyclopentane : CCC1CCCC1(C)C
1,1-dimethyl-3-ethylcyclopentane : CCC1CCC(C)(C)C1
1,1-dimethylbutyl acetate : CCCC(C)(C)OC(C)=O
1,1-dimethylcyclobutane : CC1(C)CCC1
1,1-dimethylcyclohexane : CC1(C)CCCCC1
1,1-dimethylcyclopentane : CC1(C)CCCC1
1,1-dimethylcyclopropane : CC1(C)CC1
1,1-dimethylpropyl 3-methylbutanoate : CCC(C)(C)OC(=O)CC(C)C
1,1-dimethylpropyl butanoate : CCCC(=O)OC(C)(C)CC
1,1-dimethylpropyl formate : CCC(C)(C)OC=O
1,1-dimethylpropyl propanoate : CCC(=O)OC(C)(C)CC
1,1-dimethylpropylamine : CCC(C)(C)N
1,1-diphenyl-1-butene : CCC=C(c1ccccc1)c2ccccc2
1,1-diphenyl-1-decene : CCCCCCCCC=C(c1ccccc1)c2ccc

1,2-bis(ethylthio)ethane : CCSCCSCC
1,2-bis(methylthio)ethane : CSCCSC
1,2-butadiene : C[CH]=[C]=[CH2]
1,2-butanediol : CCC(O)CO
1,2-dibromo-2,3-dimethylbutane : CC(C)C(C)(Br)CBr
1,2-dibromo-2-methylbutane : CCC(C)(Br)CBr
1,2-dibromo-2-methylpropane : CC(C)(Br)CBr
1,2-dibromo-3,3-dimethylbutane : CC(C)(C)C(Br)CBr
1,2-dibromo-3-methylbutane : CC(C)C(Br)CBr
1,2-dibromobutane : CCC(Br)CBr
1,2-dibromoethane : BrCCBr
1,2-dibromoheptane : CCCCCC(Br)CBr
1,2-dibromohexane : CCCCC(Br)CBr
1,2-dibromononane : CCCCCCCC(Br)CBr
1,2-dibromooctane : CCCCCCC(Br)CBr
1,2-dibromopentane : CCCC(Br)CBr
1,2-dibromopropane : CC(Br)CBr
1,2-dibromotetrafluoroethane : FC(F)(Br)C(F)(F)Br
1,2-dichloro-1,1,2,2-tetrafluoroethane : FC(F)(Cl)C(F)(F)Cl
1,2-dichloro-1,1,2-trifluoroethane : FC(Cl)C(F)(F)Cl
1,2-dichloro-1,1-difluoroethane : FC(F)(Cl)CCl
1,2-dichloro-1,2,3,3,4,4-hexafluorocyclobutane : FC1(F)C(F)(F)C(F)(Cl)C1(F)Cl
1,2-dichloro-1,2-difluoroethane : FC(Cl)C(F)Cl
1,2-dichloro-1-fluoroethane : FC(Cl)CCl
1,2-di

1,3-diphenyl-1-propene : C(C=Cc1ccccc1)c2ccccc2
1,3-diphenyl-2-methylpropane : CC(Cc1ccccc1)Cc2ccccc2
1,3-diphenylbutane, (±) did not work
1,3-diphenylpentane : CCC(CCc1ccccc1)c2ccccc2
1,3-diphenylpropane : C(Cc1ccccc1)Cc2ccccc2
1,3-diphenyltriazene : N(N=Nc1ccccc1)c2ccccc2
1,3-eicosanediol : CCCCCCCCCCCCCCCCCC(O)CCO
1,3-hexadiene, cis and trans did not work
1,3-nonadecanediol : CCCCCCCCCCCCCCCCC(O)CCO
1,3-pentadiene; (cis+trans) did not work
1,3-propanediamine : NCCCN
1,3-propanediol : OCCCO
1,3-propylene oxide : C1COC1
1,4,5,8-tetramethylnaphthalene : Cc1ccc(C)c2c(C)ccc(C)c12
1,4,5-trimethylnaphthalene : Cc1ccc(C)c2c(C)cccc12
1,4,6,7-tetramethylnaphthalene : Cc1ccc(C)c2cc(C)c(C)cc12
1,4,6-trimethylnaphthalene : Cc1ccc2c(C)ccc(C)c2c1
1,4-benzenedicarboxaldehyde : O=Cc1ccc(C=O)cc1
1,4-butanediamine : NCCCCN
1,4-cyclohexadiene : C1C=CCC=C1
1,4-cyclohexanedicarboxylic acid : OC(=O)C1CCC(CC1)C(O)=O
1,4-cyclooctadiene : C1CC=CCC=CC1
1,4-di-tert-butylbenzene : CC(C)(C)c1ccc(cc1)C(C)(C)C
1,4

...

isobutyl isobutyrate : CC(C)COC(=O)C(C)C
isobutyl isocyanate : CC(C)CN=C=O
isobutyl isocyanide : CC(C)C[N+]#[C-]
isobutyl mercaptan : CC(C)CS
isobutyl methacrylate : CC(C)COC(=O)C(C)=C
isobutyl pentanoate : CCCCC(=O)OCC(C)C
isobutyl propanoate : CCC(=O)OCC(C)C
isobutyl vinyl ether : CC(C)COC=C
isobutylamine : CC(C)CN
isobutylbenzene : CC(C)Cc1ccccc1
isobutylcyclohexane : CC(C)CC1CCCCC1
isobutylcyclopentane : CC(C)CC1CCCC1
isobutyraldehyde : CC(C)C=O
isobutyric acid : CC(C)C(O)=O
isobutyronitrile : CC(C)C#N
isodecanal : CC(C)CCCCCCC=O
isodecanol : CC(C)CCCCCCCO
isooctanol : CC(C)CCCCCO
isopentyl 2-methylpropanoate : CC(C)CCOC(=O)C(C)C
isopentyl acetate : CC(C)CCOC(C)=O
isopentyl formate : CC(C)CCOC=O
isopentyl hexanoate : CCCCCC(=O)OCCC(C)C
isopentyl isovalerate : CC(C)CCOC(=O)CC(C)C
isopentyl pentanoate : CCCCC(=O)OCCC(C)C
isopentyl propanoate : CCC(=O)OCCC(C)C
isopentylcyclohexane : CC(C)CCC1CCCCC1
isophorone : CC1=CC(=O)CC(C)(C)C1
isophorone diisocyanate : CC1(C)CC(CC(C)(CN=C=O)C1)N=

methylcyclopentane : CC1CCCC1
methylcyclopropane : CC1CC1
methyldecylamine : CCCCCCCCCCNC
methyldiethylamine : CCN(C)CC
methyldiisopropylamine : CC(C)N(C)C(C)C
methyldioctylamine : CCCCCCCCN(C)CCCCCCCC
methyldipropylamine : CCCN(C)CCC
methyldocosylamine : CCCCCCCCCCCCCCCCCCCCCCNC
methyldodecylamine : CCCCCCCCCCCCNC
methyleicosylamine : CCCCCCCCCCCCCCCCCCCCNC
methylethanolamine : CC(O)CN
methylethyl-sec-butylamine : CCC(C)N(C)CC
methylethyl-tert-butylamine : CCN(C)C(C)(C)C
methylethylamine : CCNC
methylethylbutylamine : CCCCN(C)CC
methylethylisobutylamine : CCN(C)CC(C)C
methylethylisopropylamine : CCN(C)C(C)C
methylethylpropylamine : CCCN(C)CC
methylglutaronitrile : CC(CCC#N)C#N
methylheneicosylamine : CCCCCCCCCCCCCCCCCCCCCNC
methylheptadecylamine : CCCCCCCCCCCCCCCCCNC
methylheptylamine : CCCCCCCNC
methylhexadecylamine : CCCCCCCCCCCCCCCCNC
methylhexylamine : CCCCCCNC
methylisobutylamine : CNCC(C)C
methylisopropylamine : CNC(C)C
methylnaphthalene : Cc1cccc2ccccc12
methylnonadecylamine : 

pentadecylamine : CCCCCCCCCCCCCCCN
pentadecylbenzene : CCCCCCCCCCCCCCCc1ccccc1
pentadecylcyclohexane : CCCCCCCCCCCCCCCC1CCCCC1
pentadecylcyclopentane : CCCCCCCCCCCCCCCC1CCCC1
pentaethylbenzene : CCc1cc(CC)c(CC)c(CC)c1CC
pentafluoroethane : FC(F)C(F)(F)F
pentafluoroethyl trifluorovinyl ether : FC(F)=C(F)OC(F)(F)C(F)(F)F
pentamethylbenzene : Cc1cc(C)c(C)c(C)c1C
pentamethylphenol : Cc1c(C)c(C)c(O)c(C)c1C
pentanal : CCCCC=O
pentanedial : O=CCCCC=O
pentanoic acid : CCCCC(O)=O
pentanol : CCCCCO
pentyl acetate : CCCCCOC(C)=O
pentyl butanoate : CCCCCOC(=O)CCC
pentyl formate : CCCCCOC=O
pentyl heptanoate : CCCCCCC(=O)OCCCCC
pentyl hexanoate : CCCCCOC(=O)CCCCC
pentyl mercaptan : CCCCCS
pentyl octanoate : CCCCCCCC(=O)OCCCCC
pentyl pentanoate : CCCCCOC(=O)CCCC
pentyl propanoate : CCCCCOC(=O)CC
pentylamine : CCCCCN
pentylbenzene : CCCCCc1ccccc1
pentylcyclohexane : CCCCCC1CCCCC1
pentylcyclopentane : CCCCCC1CCCC1
perchloric acid : O[Cl](=O)(=O)=O
perchloryl fluoride : F[Cl](=O)(=O)=O
perfluoroethyl m

tetracosylamine : CCCCCCCCCCCCCCCCCCCCCCCCN
tetradecamethylhexasiloxane : C[Si](C)(C)O[Si](C)(C)O[Si](C)(C)O[Si](C)(C)O[Si](C)(C)O[Si](C)(C)C
tetradecanal : CCCCCCCCCCCCCC=O
tetradecane : CCCCCCCCCCCCCC
tetradecanenitrile : CCCCCCCCCCCCCC#N
tetradecanoic acid : CCCCCCCCCCCCCC(O)=O
tetradecanol; mixed isomers did not work
tetradecylamine : CCCCCCCCCCCCCCN
tetradecylcyclohexane : CCCCCCCCCCCCCCC1CCCCC1
tetradecylcyclopentane : CCCCCCCCCCCCCCC1CCCC1
tetraethyl lead : CC[Pb](CC)(CC)CC
tetraethylene glycol : OCCOCCOCCOCCO
tetraethylene glycol dimethyl ether : COCCOCCOCCOCCOC
tetraethylsilane : CC[Si](CC)(CC)CC
tetrafluorodimethyl ether did not work
tetrafluoroethylene : FC(F)=C(F)F
tetrafluorohydrazine : FN(F)N(F)F
tetrahydrofuran : C1CCOC1
tetrahydrofurfuryl alcohol : OCC1CCCO1
tetrahydroneral did not work
tetrahydrothiophene : C1CCSC1
tetrakis(trimethylsilyloxy)silane : C[Si](C)(C)O[Si](O[Si](C)(C)C)(O[Si](C)(C)C)O[Si](C)(C)C
tetramethylsilane : C[Si](C)(C)C
tetranitromethane : [O-][N+](=

trans-5-undecene : CCCCC/C=C/CCCC
trans-7-tetradecene : CCCCCC\C=C\CCCCCC
trans-cinnamic acid : OC(=O)\C=C\c1ccccc1
trans-crotonaldehyde : C/C=C/C=O
trans-crotonitrile did not work
trans-decahydronaphthalene : C1CC[C@@H]2CCCC[C@H]2C1
trans-dicyano-1-butene did not work
trans-stilbene : c1ccc(cc1)\C=C\c2ccccc2
tributyl borate : CCCCOB(OCCCC)OCCCC
tributylamine : CCCCN(CCCC)CCCC
trichloroacetaldehyde : ClC(Cl)(Cl)C=O
trichloroacetic acid : OC(=O)C(Cl)(Cl)Cl
trichloroacetyl chloride : ClC(=O)C(Cl)(Cl)Cl
trichlorobenzene : Clc1cccc(Cl)c1Cl
trichloroethylene : ClC=C(Cl)Cl
trichloroethylsilane : [SiH3]CC(Cl)(Cl)Cl
trichlorofluoromethane : FC(Cl)(Cl)Cl
trichlorophenylsilane : Cl[Si](Cl)(Cl)c1ccccc1
trichlorovinylsilane : [SiH3]C(Cl)=C(Cl)Cl
tricosane : CCCCCCCCCCCCCCCCCCCCCCC
tricosylamine : CCCCCCCCCCCCCCCCCCCCCCCN
tricyclo[3.3.1.13,7]decane : C1C2CC3CC1CC(C2)C3
tridecane : CCCCCCCCCCCCC
tridecanenitrile : CCCCCCCCCCCCC#N
tridecanoic acid : CCCCCCCCCCCCC(O)=O
tridecyl formate : CCCCCCCCCCCCC

In [8]:
data_SMILES.to_csv('boiling_data_smiles.csv',index=False)

In [10]:
len(data_SMILES[data_SMILES['SMILES'] == '-'])
#386 Failed Chemicals

386

#### Method 2: Using PubChemPy

In [14]:
import pubchempy as pcp

Playing around with the package:

In [15]:
results_aspirin = pcp.get_compounds('Aspirin','name')
results_pinene = pcp.get_compounds('(+)-a-pinene','name')

In [16]:
print(results_aspirin)
print(results_aspirin[0].isomeric_smiles)
#Empty list suggests no results were found.
#Try running all 6000 chemicals into this and find all chemicals with no result

[Compound(2244)]
CC(=O)OC1=CC=CC=C1C(=O)O


In [ ]:
#If pubchempy method is to be used for whole data set, this block will be used.
failed_pubchempy = []
SMILES_pubchempy = []
more_than_1 = []

for chemical in boiling_data['name']:
    res = pcp.get_compounds(chemical,'name')

    if len(res) == 1:
        SMILES_pubchempy.append([chemical, res[0].isomeric_smiles])
        print(f'{chemical} : {res[0].isomeric_smiles}')
    elif len(res) == 0:
        failed_pubchempy.append(chemical)
        print(f'{chemical} failed!')
    else:      
        more_than_1.append(chemical)
        print(f'{chemical}: Found more than 1 result')
print(len(more_than_1))
print(len(failed_pubchempy))

In [23]:
def get_SMILES_pubchempy(chemical):
    res = pcp.get_compounds(chemical,'name')

    try:
        print(f'{chemical} : {res[0].isomeric_smiles}')
        return str(res[0].isomeric_smiles)
    except:
        print(f'{chemical} failed!')
        return '-'

#### Now: Combine the 2 methods together
I took the failed chemicals list from CIR and then tried to put them through pubchempy to maximise the number of chemicals I can work with

In [18]:
data_SMILES = pd.read_csv('boiling_data_smiles.csv')

In [20]:
failed_indices = data_SMILES.index[data_SMILES['SMILES'] == '-'].tolist()

In [24]:
len(failed_indices)

386

In [29]:
failed_chemicals = data_SMILES.iloc[failed_indices]['name']

In [30]:
failed_chemicals

0             (+)-a-pinene
2             (-)-a-pinene
3        (-)-b-citronellol
21       (1S)-(-)-b-pinene
28        (<+->)-2-butanol
               ...        
5958    triethylene glycol
6011              vanillin
6027               xylenes
6028     zirconium bromide
6030      zirconium iodide
Name: name, Length: 386, dtype: object

In [33]:
for i,chemical in enumerate(failed_chemicals):
    fail_index = failed_indices[i]
    data_SMILES.at[fail_index, 'SMILES']=get_SMILES_pubchempy(chemical)

(+)-a-pinene failed!
(-)-a-pinene : CC1=CC[C@H]2C[C@@H]1C2(C)C
(-)-b-citronellol failed!
(1S)-(-)-b-pinene failed!
(<+->)-2-butanol failed!
(<+->)-2-chlorooctane failed!
(R)-(+)-b-citronellol : C[C@H](CCC=C(C)C)CCO
(±)-a-pinene failed!
(±)-b-citronellol failed!
(±)-glycidol failed!
1,1',1''-(1-ethanyl-2-ylidene)trisbenzene failed!
1,1,3-trimethylcyclopentane : CC1CCC(C1)(C)C
1,1,4-tribromobutane : C(CC(Br)Br)CBr
1,1,4-trichlorobutane : C(CC(Cl)Cl)CCl
1,1,4-trifluorobutane : C(CC(F)F)CF
1,1,4-triiodobutane : C(CC(I)I)CI
1,1,4-trimethylcyclohexane : CC1CCC(CC1)(C)C
1,1-diphenylundecane : CCCCCCCCCCC(C1=CC=CC=C1)C2=CC=CC=C2
1,11-dibromoundecane : C(CCCCCBr)CCCCCBr
1,12-dibromododecane : C(CCCCCCBr)CCCCCBr
1,2,3-benzenetriol : C1=CC(=C(C(=C1)O)O)O
1,2,4-trichloro-5-methylbenzene : CC1=CC(=C(C=C1Cl)Cl)Cl
1,2-dichloro-1,2,3,3,4,4-hexafluorocyclobutane : C1(C(C(C1(F)Cl)(F)Cl)(F)F)(F)F
1,2-dichloro-3-methylbutane : CC(C)C(CCl)Cl
1,2-diiodobutane : CCC(CI)I
1,2-dimethylpropylamine : CC(C)C(C)N


3,4-dimethyl-4-octanol failed!
3,5-diethylheptane : CCC(CC)CC(CC)CC
3,5-dimethyl-3-hexanol : CCC(C)(CC(C)C)O
3,5-dimethyl-3-hexanol, (±) failed!
3,6-diethyloctane : CCC(CC)CCC(CC)CC
3,6-dimethyl-4-ethyloctane failed!
3,7-dimethyl-1-octanol, (±) failed!
3,7-dimethyl-1-octene : CC(C)CCCC(C)C=C
3,7-dimethyl-2-octanol : CC(C)CCCC(C)C(C)O
3,7-dimethyl-3-octanol : CCC(C)(CCCC(C)C)O
3,7-dimethyl-3-octanol, (±) failed!
3,7-dimethyldecane : CCCC(C)CCCC(C)CC
3,7-dimethylnonane : CCC(C)CCCC(C)CC
3,7-dimethyloctanyl acetate : CC(C)CCCC(C)CCOC(=O)C
3,7-dimethyloctanyl butyrate : CCCC(=O)OCCC(C)CCCC(C)C
3,7-dimethylquinoline : CC1=CC2=C(C=C1)C=C(C=N2)C
3,8-dimethyldecane : CCC(C)CCCCC(C)CC
3,8-dimethylquinoline : CC1=C2C(=CC=C1)C=C(C=N2)C
3,cis-4-dimethylcyclohexene failed!
3,cis-4-dimethylcyclopentene failed!
3,cis-4-dimethylthiacyclohexane failed!
3,cis-4-dimethylthiacyclopentane failed!
3-aminoheptane : CCCCC(CC)N
3-aminohexane : CCCC(CC)N
3-chloro-2-methyl-1-pentene : CCC(C(=C)C)Cl
3-chloro-2-me

In [34]:
len(data_SMILES[data_SMILES['SMILES'] == '-'])

123

Now only 123 did not return a SMILES value!

In [35]:
data_SMILES.to_csv('boiling_data_smiles.csv',index=False)